In [1]:
import torch
from torch.nn import DataParallel
from torch.utils.data import DataLoader
from torch.optim import AdamW

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import os
import gc
from tqdm import tqdm
import sys
sys.path.append("../src")
from config import read_config, update_config
from metric import cal_mae_metric
from dataset import read_data
from util import smart_avg

In [2]:
config = read_config("Base")
config.gpu = [0]
config = update_config(config)

Using device: cuda
Number of device: 1
Model Output Folder: /home/vincent/Kaggle/GBVPP/output/Base/


In [3]:
subs = {
    "LSTM5_REG_PL": "../output/LSTM5_REG_PL/test_pred_all_18832.csv",
    "Public_PulpFriction": "../output/Public_PulpFriction/test_pred_all_18507.csv",
    "LSTM5_CLS_DO02_CH01_OP01_physics_SiLU":"../output/LSTM5_CLS_DO02_CH01_OP01_physics_SiLU/test_pred_all_18035.csv"
}
ensemble_folder = "ensemble_1031"
if not os.path.exists(config.output_folder + "/" + ensemble_folder):
    ensemble_folder = config.output_folder + "/" + ensemble_folder
    os.makedirs(ensemble_folder)

In [4]:
train, test = read_data(config)

In [5]:
pressure_unique = np.load("/home/vincent/Kaggle/GBVPP/input/pressure_unique.npy")

In [6]:
sub_all = None
for key, sub_file in tqdm(subs.items()):
    train[key] = 0
    tmp = pd.read_csv(sub_file)
    if sub_all is None:
        sub_all = tmp[["id","pressure"]].copy()
        sub_all["pressure"] = 0
        
    for fold in range(5):
        if "Public_PulpFriction" == key:
            train.loc[train.query(f"fold=={fold}").index, key] = np.load(f'../output/{key}/oof{fold}.npy').flatten()
        else:
            train.loc[train.query(f"fold=={fold}").index, key] = torch.load(f'../output/{key}/Fold_{fold}_best_model.pth')['valid_preds'].flatten()
        sub_all[key+f"_{fold}"] = tmp[f"preds_fold{fold}"]
        
train["pressure_smart"] = smart_avg(train[list(subs.keys())], axis=1)
train["pressure_smart"] = train["pressure_smart"].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])
CVs = train.groupby("fold").apply(lambda df: cal_mae_metric(df["pressure"],df["pressure_smart"],1-df["u_out"]))
CV = np.mean(CVs)
print("Ensemble CV Median: Mean {:.4f}, std {:.4f}".format(np.mean(CVs), np.std(CVs)))

for fold in tqdm(range(5)):
    sub_keys = [key+f"_{fold}" for key in subs.keys()]
    test_fold_pred = smart_avg(sub_all[sub_keys], axis=1)
    sub_fold = sub_all[["id"]].copy()
    sub_fold[f'preds_fold{fold}'] = test_fold_pred
    sub_fold[f'preds_fold{fold}'] = sub_fold[f'preds_fold{fold}'].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()]) 
    sub_fold.to_csv(ensemble_folder + f"/test_fold{fold}.csv",index=False)

preds_cols = [key+f"_{i}"  for key in subs.keys() for i in range(5)]
sub_all["pressure"] = smart_avg(sub_all[preds_cols], axis=1)
sub_all["pressure"] = sub_all["pressure"].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])       
sub_all.to_csv(ensemble_folder + f"/sub_all_{CV*1e5:.0f}.csv")
sub_all[["id","pressure"]].to_csv(ensemble_folder + f"/submission_{CV*1e5:.0f}.csv",index=False)
print(sub_all.shape, train.shape)
sub_all

100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.33s/it]


Inliers:  2062271 -> compute mean
Outliers: 3973729 -> compute median
Total:    6036000


  0%|                                                                                     | 0/5 [00:00<?, ?it/s]

Inliers:  1364347 -> compute mean
Outliers: 2659653 -> compute median
Total:    4024000


 20%|███████████████▍                                                             | 1/5 [00:17<01:09, 17.43s/it]

...

 80%|█████████████████████████████████████████████████████████████▌               | 4/5 [01:09<00:17, 17.22s/it]

Inliers:  1371654 -> compute mean
Outliers: 2652346 -> compute median
Total:    4024000


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [01:26<00:00, 17.24s/it]


Ensemble CV Median: Mean 0.1625, std 0.0018
Inliers:  1052168 -> compute mean
Outliers: 2971832 -> compute median
Total:    4024000
(4024000, 17) (6036000, 13)


,id,pressure,LSTM5_REG_PL_0,LSTM5_REG_PL_1,LSTM5_REG_PL_2,LSTM5_REG_PL_3,LSTM5_REG_PL_4,Public_PulpFriction_0,Public_PulpFriction_1,Public_PulpFriction_2,Public_PulpFriction_3,Public_PulpFriction_4,LSTM5_CLS_DO02_CH01_OP01_physics_SiLU_0,LSTM5_CLS_DO02_CH01_OP01_physics_SiLU_1,LSTM5_CLS_DO02_CH01_OP01_physics_SiLU_2,LSTM5_CLS_DO02_CH01_OP01_physics_SiLU_3,LSTM5_CLS_DO02_CH01_OP01_physics_SiLU_4
0,1,6.259305,6.189002,6.189002,6.189002,6.329606,6.259304,6.322844,6.317010,6.284339,6.313703,6.378181,6.259305,6.259305,6.259305,6.259305,6.259305
1,2,5.978096,5.907794,5.978096,6.048398,5.978096,6.048398,5.960816,5.904262,5.921993,5.857764,6.009814,5.978096,5.978096,5.978096,5.978096,5.907794
2,3,7.102930,7.102930,7.173232,7.173232,7.102930,7.102930,7.150896,7.129235,7.110208,7.096230,7.139535,7.032628,7.102930,7.102930,7.032628,7.102930
3,4,7.665347,7.595045,7.665347,7.595045,7.665347,7.665347,7.696169,7.636521,7.671168,7.581084,7.704820,7.595045,7.595045,7.665347,7.665347,7.665347
4,5,9.141693,9.141692,9.141692,9.141692,9.141692,9.071390,9.168344,9.169226,9.156463,9.098163,9.211409,9.141693,9.071390,9.071390,9.141693,9.071390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,4023996,6.189002,0.002414,0.002414,0.002414,0.002414,0.002414,6.101240,6.134991,6.196092,6.196517,6.186067,10.547735,14.976771,18.280971,17.226439,13.641030
4023996,4023997,6.259305,0.002414,0.002414,0.002414,0.002414,0.002414,6.221967,6.426729,6.297765,6.257534,6.277758,10.547735,15.328281,17.859159,16.874929,13.641030
4023997,4023998,6.470211,0.002414,0.002414,0.002414,0.002414,0.002414,6.323777,6.279234,6.436971,6.474350,6.447019,10.688340,16.242209,17.718554,21.796079,13.641030
4023998,4023999,6.189002,0.002414,0.002414,0.002414,0.002414,0.002414,6.026303,6.170699,6.155049,6.171207,6.170768,11.110153,17.718554,17.718554,21.796079,13.992541


In [15]:
sub_pub = pd.read_csv("../output/submission_1362.csv")

In [16]:
sub_all["pressure"] = ((sub_all["pressure"] + sub_pub["pressure"])/2).map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])

In [17]:
sub_all[["id","pressure"]].to_csv(ensemble_folder + f"/submission_{CV*1e5:.0f}_with_pub.csv",index=False)